In [21]:
import os
#from change_extensions import *
#from baseline import *
#from data_cleaning import *
#from extract_stories import *
#from testing import *
#import Data
#Data/cnn/stories
%run change_extensions Data/cnn/stories .story .txt

Changing all files in /Data/cnn/stories with a .story extension to use a .txt extension instead...
92579 files in directory


In [26]:
%run data_cleaning.py

There are 92579 files
Finished 2314 files: 2314/92579 = 2.499486924680543%, in 148.23329901695251 seconds
Finished 4628 files: 4628/92579 = 4.998973849361086%, in 144.02052474021912 seconds
Finished 6942 files: 6942/92579 = 7.49846077404163%, in 142.00053310394287 seconds
Finished 9256 files: 9256/92579 = 9.997947698722172%, in 142.37709403038025 seconds
Finished 11570 files: 11570/92579 = 12.497434623402716%, in 148.56068086624146 seconds
Finished 13884 files: 13884/92579 = 14.99692154808326%, in 149.5020089149475 seconds
Finished 16198 files: 16198/92579 = 17.4964084727638%, in 161.79482650756836 seconds
Finished 18512 files: 18512/92579 = 19.995895397444343%, in 158.63236451148987 seconds
Finished 20826 files: 20826/92579 = 22.49538232212489%, in 160.34246850013733 seconds
Finished 23140 files: 23140/92579 = 24.994869246805433%, in 152.88791394233704 seconds
Finished 25454 files: 25454/92579 = 27.494356171485975%, in 157.29456281661987 seconds
Finished 27768 files: 27768/92579 = 29.

In [116]:
import spacy
import os
import csv
import numpy as np
#import nltk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

nlp = spacy.load("en_core_web_sm")

#doc will be in csv form, gets labels and raw sentences
def get_doc_raw_data(lab_doc_path):
    sentences = []
    labels = []
    with open(lab_doc_path, "r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        for line in reader:
            sentence = line["sentence"]
            label = line["label"]
            sentences.append(sentence)
            labels.append(label)
    return [sentences, labels]         
            
def get_doc_LR_inputs(lab_doc_path):
    doc_raw_data = get_doc_raw_data(lab_doc_path)
    raw_sents = doc_raw_data[0]
    raw_labels = doc_raw_data[1]
    vector_sents_list = [nlp(sent).vector.reshape(1,-1) for sent in raw_sents]
    vector_sents_matrix = np.concatenate(vector_sents_list)
    return [raw_labels, vector_sents_matrix]

#testing on single doc
def LR_on_doc(lab_doc_path):
    inputs = get_doc_LR_inputs(lab_doc_path)
    labels = inputs[0]
    sents_matrix = inputs[1]
    X_train, X_test, y_train, y_test = train_test_split(sents_matrix, labels, random_state=42)
    LR = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', fit_intercept=False)
    LR.fit(X_train, y_train)
    pred_labels = LR.predict(X_test)
    actual_labels = y_test
    overall_test_results = []
    for i in range(len(pred_labels)):
        overall_test_results.append({'Number': i, 'pred': pred_labels[i], 'act': actual_labels[i]})
    for result in overall_test_results:
        print(result)
        print()
        
def LR_on_first_n(dir_name, n, train_set):
    raw_file_names = os.listdir(dir_name)[0:n]
    LR_x_inputs = []
    LR_labels = []
    
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    c = 0
    for raw_name in raw_file_names:
        file_path = os.path.join("labeled_data2_0.8", raw_name)
        doc_LR_inputs = get_doc_LR_inputs(file_path)
        LR_x_inputs.append(doc_LR_inputs[1])
        LR_labels.append(doc_LR_inputs[0])
        if c in train_set:
            X_train.append(doc_LR_inputs[1])
            y_train.append(doc_LR_inputs[0])
        else:
            X_test.append(doc_LR_inputs[1])
            y_test.append(doc_LR_inputs[0])
        c += 1
        
    #X_train, X_test, y_train, y_test = train_test_split(LR_x_inputs, LR_labels, random_state=42)
    
    
    X_train_list = []
    for doc_matrix in X_train:
        for sent_vector in doc_matrix:
            X_train_list.append(sent_vector)
    X_train_matrix = X_train_list
    #X_train_matrix = np.concatenate(X_train_list)
    y_train_list = []
    for doc in y_train:
        for label in doc:
            y_train_list.append(label)
    LR = LogisticRegression(multi_class='multinomial', fit_intercept=False, solver='lbfgs', penalty='l2')
    LR.fit(X_train_matrix, y_train_list)
    pred_labels = []
    act_labels = []
    num_sentences = 0
    num_act_true = 0
    num_pred_true = 0
    fp = 0
    fn = 0
    tp = 0
    tn = 0
    
    for i in range(len(X_test)):
        pred_labels.append(LR.predict(X_test[i]))
        act_labels.append(y_test[i])
    for i in range(len(pred_labels)):
        #print('Next doc tests')
        otr = []
        for c in range(len(pred_labels[i])):
            curr_pred = pred_labels[i][c]
            curr_act = act_labels[i][c]
            if(curr_pred == 'True' and curr_act == 'True'):
                tp += 1
            elif(curr_pred == 'True' and curr_act == 'False'):
                fp += 1
            elif(curr_pred == 'False' and curr_act == 'True'):
                fn += 1
            elif(curr_pred == 'False' and curr_act == 'False'):
                tn += 1
            if(curr_pred == 'True'):
                num_pred_true += 1
            if(curr_act == 'True'):
                num_act_true += 1
            num_sentences += 1
            otr.append({'Number': c, 'pred': curr_pred, 'act': curr_act})
        #for res in otr:
            #print(res)
        #print()
    #coefs = LR.coef_
    #print(coefs[0])
    print(num_sentences)
    print(num_act_true)
    print(num_pred_true)
    print(fp)
    print(fn)
    print(tp)
    print(tn)
    return [pred_labels, act_labels, LR.coef_]
#def get_train_set()

In [121]:
import os
import math
n = 22651
split_index = math.floor(.75 * n)
dir_name = "labeled_data2_0.8"
raw_name = os.listdir(dir_name)[100]
#story_file = os.path.join("stories", raw_name + ".txt")
labeled_file = os.path.join("labeled_data2_0.8", raw_name)
#from Log_Reg_Rel_Functions import *
#LR_on_doc(labeled_file)
train_set = []

for i in range(split_index):
    train_set.append(i)

results = LR_on_first_n(dir_name, n, train_set)
pred_labels = results[0]
act_labels = results[1]
coefs = results[2]

185567
7277
19
17
7275
2
178273
